<a href="https://colab.research.google.com/github/hochthom/OG-AI4Artists-2022/blob/main/Image_Generation/StyleGAN2_ADA_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training StyleGAN2 on Colab



First we check for what GPU we have been assigned. The newer the model the faster the training will go on!

- T4, Turing, 2018, 16GB
- V100, Volta, 2017, 16GB
- P100, Pascal, 2016, 16GB
- K80, Kepler, 2014, 12GB

In [1]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-112b3b6c-fcfe-c891-3e04-eeceb3baef09)


Next we connect to Google drive account:

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls /content/drive/

MyDrive


In [3]:
!pip install ninja opensimplex torch==1.7.1 torchvision==0.8.2

     |████████████████████████████████| 108 kB 29.7 MB/s 
     |████████████████████████████████| 776.8 MB 17 kB/s 
     |████████████████████████████████| 12.8 MB 31.6 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.11.0+cu113
    Uninstalling torch-1.11.0+cu113:
      Successfully uninstalled torch-1.11.0+cu113
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.12.0+cu113
    Uninstalling torchvision-0.12.0+cu113:
      Successfully uninstalled torchvision-0.12.0+cu113
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.12.0 requires torch==1.11.0, but you have torch 1.7.1 which is incompatible.
torchaudio 0.11.0+cu113 requires torch==1.11.0, but you have torch 1.7.1 which is incompatible.


# Install StyleGAN2 Repo

In [4]:
HOME_DIR = '/content/drive/MyDrive/AI4Artists/OG-AI4Artists-2022'
!ls $HOME_DIR

'AI for Artists - Img-Gen.gslides'    notebooks
'AI for Artists - Intro.gslides'      stylegan2-ada-pytorch
'AI for Artists - Text-Gen.gslides'


In [5]:
import os

sg2_dir = os.path.join(HOME_DIR, 'stylegan2-ada-pytorch')
if os.path.isdir(sg2_dir):
    %cd $sg2_dir
elif os.path.isdir(HOME_DIR):
    #install script
    %cd $HOME_DIR
    !git clone https://github.com/hochthom/stylegan2-ada-pytorch
    %cd stylegan2-ada-pytorch
    !mkdir downloads
    !mkdir datasets
    !mkdir pretrained
    !mkdir samples
else:
  print('Set valid directory for HOME_DIR!')

/content/drive/MyDrive/AI4Artists/OG-AI4Artists-2022/stylegan2-ada-pytorch


In [ ]:
!ls

apply_factor.py		      Network_Blending_ADA_PT.ipynb
blend_models.py		      pbaylies_projector.py
calc_metrics.py		      pretrained
closed_form_factorization.py  projector.py
combine_npz.py		      __pycache__
datasets		      README.md
dataset_tool.py		      results
dnnlib			      samples
Dockerfile		      SG2-ADA-PT_AudioReactive+Pitch.ipynb
docker_run.sh		      SG2_ADA_PT_to_Rosinality.ipynb
docs			      SG2_ADA_PyTorch.ipynb
downloads		      StyleCLIP_playground.ipynb
export_weights.py	      StyleGAN2_CLIP_approach_v1.ipynb
flesh_digression.py	      style_mixing.py
generate.py		      torch_utils
legacy.py		      training
LICENSE.txt		      train.py
metrics			      util


# Dataset Preparation
Upload your images in one zip file to the dataset folder in your local repo (the one we just generated)

# Model Training
First we have to set some variables ...

* `dataset_path`: this is the path to your .zip file
* `resume_from`: if you’re starting a new dataset I recommend `'ffhq256'` or `'./pretrained/wikiart.pkl'`


In [7]:
#required: definitely edit these!
#dataset_path = './datasets/metFaces_256.zip'
dataset_path = './datasets/abstract_512.zip'
#resume_from = 'noresume'  # 'ffhq256'
resume_from = './results/00005-abstract_512-mirror-auto1-gamma10-bg-noresume/network-snapshot-000064.pkl'
mirror_x = True
mirror_y = False

#optional: you might not need to edit these
gamma = 10.0
augs = 'bg'
#config = '11gb-gpu'  # for 512x512 images or greater on a K80
config = 'auto'      # otherwise (and for 256x256 images)
snapshot = 4

# for resuming training set this values accordingly
aug_strength = 0.094
train_count = 64

To see all available parameters for training you can execute the following code

In [ ]:
!python train.py --help

In [ ]:
!python train.py --gpus=1 --cfg=auto --outdir=./results --data=$dataset_path --snap=$snapshot --resume=$resume_from --metrics=None --augpipe=$augs --gamma=$gamma --mirror=$mirror_x --mirrory=$mirror_y --initstrength=$aug_strength --nkimg=$train_count


Training options:
{
  "num_gpus": 1,
  "image_snapshot_ticks": 4,
  "network_snapshot_ticks": 4,
  "metrics": [],
  "random_seed": 0,
  "training_set_kwargs": {
    "class_name": "training.dataset.ImageFolderDataset",
    "path": "./datasets/abstract_512.zip",
    "use_labels": false,
    "max_size": 1000,
    "xflip": true,
    "resolution": 512
  },
  "data_loader_kwargs": {
    "pin_memory": true,
    "num_workers": 3,
    "prefetch_factor": 2
  },
  "G_kwargs": {
    "class_name": "training.networks.Generator",
    "z_dim": 512,
    "w_dim": 512,
    "mapping_kwargs": {
      "num_layers": 2
    },
    "synthesis_kwargs": {
      "channel_base": 32768,
      "channel_max": 512,
      "num_fp16_res": 4,
      "conv_clamp": 256
    }
  },
  "D_kwargs": {
    "class_name": "training.networks.Discriminator",
    "block_kwargs": {},
    "mapping_kwargs": {},
    "epilogue_kwargs": {
      "mbstd_group_size": 4
    },
    "channel_base": 32768,
    "channel_max": 512,
    "num_fp16_res"